In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
from rnacomp.fastai_fit import *
from rnacomp.dataset import RNA_DatasetV1, LenMatchBatchSampler, DeviceDataLoader
from rnacomp.models import RNA_ModelV2
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

In [6]:
class CFG:
    path = Path("../data/split")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 64
    num_workers = 16
    device = 'cuda'
    seed = 2023
    out = 'exp_00'
    
    dim = 192
    depth = 12
    dim_head = 32
    
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [7]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
df_train = pd.read_parquet(CFG.path / f"train_data_{CFG.split_id}.parquet")
df_valid = pd.read_parquet(CFG.path / f"valid_data_{CFG.split_id}.parquet")
df_train = df_train.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)
df_valid = df_valid.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)

ds_train = RNA_DatasetV1(df_train)
ds_train_len = RNA_DatasetV1(df_train, mask_only=True)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = RNA_DatasetV1(df_valid)
ds_val_len = RNA_DatasetV1(df_valid,mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=True)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

In [ ]:
learn = Learner(data,
                RNA_ModelV2().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,every_epoch=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

epoch,train_loss,valid_loss,mae,time
0,0.140216,0.157737,0.195171,16:47
